In [ ]:
from imports import *
from functions_uc import *

def timellm_predict(full_data, scaler, prediction_length, context_length, prompt_prefix, learning_rate, batch_size, max_steps):

    full_data_test = full_data.groupby("id").tail(prediction_length)
    full_data_train = full_data.drop(full_data_test.index)

    full_data_train_ = full_data_train.rename(columns={'date': 'ds', 'id': 'unique_id', 'value': 'y'}).drop(columns=['type'])
    full_data_test_ = full_data_test.rename(columns={'date': 'ds', 'id': 'unique_id', 'value': 'y'}).drop(columns=['type'])
    full_data = full_data.rename(columns={'date': 'ds', 'id': 'unique_id', 'value': 'y'})

    random.seed(42)
    n_observed = full_data_test_['unique_id'].nunique()
    random_encounter_ids = random.sample(list(full_data_train_['unique_id']), n_observed)

    full_data_train_limit = full_data_train_[full_data_train_['unique_id'].isin(random_encounter_ids)]
    full_data_test_limit = full_data_test_[full_data_test_['unique_id'].isin(random_encounter_ids)]

    #TimeLLM with hyperparameters
    timellm = TimeLLM(
        h=prediction_length,
        input_size=context_length,
        llm='meta-llama/Llama-2-7b-hf',
        prompt_prefix=prompt_prefix,
        batch_size=batch_size,
        d_llm=4096,
        valid_batch_size=batch_size,
        windows_batch_size=batch_size,
        max_steps=max_steps,
        learning_rate=learning_rate
    )

    nf = NeuralForecast(models=[timellm], freq='D')
    nf.fit(df=full_data_train_limit)
    forecasts = nf.predict(full_data_train_)

    def rescale_to_original(scaled_df, scaler, columns_to_rescale):
        scaled_df[columns_to_rescale] = scaler.inverse_transform(scaled_df[columns_to_rescale])
        return scaled_df

    actuals = rescale_to_original(full_data_test_, scaler, ["y"])
    predictions = rescale_to_original(forecasts, scaler, ["TimeLLM"])

    return predictions, actuals

def objective(trial):
    # hyperparameter search space
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    max_steps = trial.suggest_int('max_steps', 50, 300)
    attention_dropout = trial.suggest_uniform('attention_dropout', 0.0, 0.5)
    n_heads = trial.suggest_categorical('n_heads', [4, 8, 16, 32])
    d_llm = trial.suggest_categorical('d_llm', [1024, 2048, 4096])


    predictions, actuals = timellm_predict(
        full_data=your_full_data,
        scaler=scaler_ume,
        prediction_length=prediction_length,
        context_length=context_length,
        prompt_prefix=prompt_prefix,
        learning_rate=learning_rate,
        attention_dropout=attention_dropout,
        n_heads=n_heads,
        max_steps=max_steps
        batch_size=24,
        d_llm=d_llm,
        valid_batch_size=24,
        windows_batch_size=24,
    )

    # Compute evaluation metric
    score = mean_absolute_error(actuals['y'], predictions['TimeLLM'])
    return score


def run_optuna_study(n_trials=20):
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials)
    print("Best trial:")
    print(study.best_trial)

    print("Best hyperparameters:")
    for key, value in study.best_trial.params.items():
        print(f"  {key}: {value}")
    return study


train = run_optuna_study()
